In [56]:
import re
import os

def extract_data_from_txt(file_path):
    """
    Extract bounding box parameters, label, and image name from a given .txt file.
    
    Parameters:
    - file_path: path to the .txt file.
    
    Returns:
    - List of tuples with format (img_name, x_min, y_min, x_max, y_max, label).
    """
    
    with open(file_path, "r") as file:
        lines = file.readlines()
    
    # Extract image name from the file path
    img_name = os.path.basename(file_path).replace("2021", "").split("_")[0]
    
    # List to store extracted data
    data = []
    h, w = map(int, lines[-1].strip().split(": ")[1].split(", "))
    for line in lines:
        if "BBOX" in line:
            # Extract bbox coordinates
            bbox = re.search(r"tensor\(\[(.+?)\]\)", line).group(1).split(", ")
            x_min, y_min, x_max, y_max = map(float, bbox)
            
            # Extract label
            label = re.search(r"LABEL: (.+?) \(ID", line).group(1)

            data.append((str(img_name), x_min, y_min, x_max, y_max, label, "\"" + str((h, w)) + "\""))
    
    return data



In [57]:
import os

directory_path = "output/from_dataset/filtered_unknown/SOWODB/"
desired_suffix = "t1.pth.txt"

# List all files in the directory
all_files = os.listdir(directory_path)

# Filter the files based on the desired suffix
filtered_files = [f for f in all_files if f.endswith(desired_suffix)]

print(len(filtered_files))


4952


In [58]:
# List to store all the extracted data from the files
all_data = []

# Extract data from all the files
for path in filtered_files:
    all_data.extend(extract_data_from_txt(directory_path + path))

# Convert the extracted data into a CSV format
csv_output_path = "bbox_data_unscaled.csv"

# Sort all_data based on the image_id before writing to the CSV file
all_data.sort(key=lambda x: int(x[0]))

# Writing the combined data to a CSV file
with open(csv_output_path, "w") as csv_file:
    # Writing the header
    csv_file.write("img_name,x_min,y_min,x_max,y_max,label, scaled_size\n")
    
    # Writing the data
    for row in all_data:
        csv_file.write(",".join(map(str, row)) + "\n")

# 
csv_output_path


'bbox_data_unscaled_gt.csv'

In [55]:
import pandas as pd
import os
from PIL import Image

def get_actual_dimensions(image_id, image_dir):
    """
    Get the actual dimensions (width, height) of an image given its image_id.

    Parameters:
    - image_id: ID/name of the image (without file extension).
    - image_dir: Directory containing the images.

    Returns:
    - Tuple (width, height) of the actual image dimensions.
    """
    image_path = os.path.join(image_dir, image_id + ".jpg")
    
    # Check if the image exists, if not, drop the first 2 digits and try again
    if not os.path.exists(image_path):
        image_id = image_id[2:]
        image_path = os.path.join(image_dir, image_id + ".jpg")
    
    with Image.open(image_path) as img:
        return img.size

def scale_bbox(row, image_dir):
    # Get actual image dimensions
    actual_width, actual_height = get_actual_dimensions(row['img_name'], image_dir)

    # Store the actual size as a tuple-style string
    row['actual_size'] = f"({actual_height}, {actual_width})"
    
    # Get scaled dimensions from the CSV
    scaled_height, scaled_width = tuple(map(int, row[' scaled_size'][1:-1].split(',')))
    
    # Compute scaling factors
    x_scale = actual_width / scaled_width
    y_scale = actual_height / scaled_height
    
    # Scale bounding box coordinates
    row['x_min'] *= x_scale
    row['y_min'] *= y_scale
    row['x_max'] *= x_scale
    row['y_max'] *= y_scale
    
    return row

image_dir = "/training_data_2/yuetian/OWOD/JPEGImages"
scaled_data =  pd.read_csv('bbox_data_unscaled.csv', dtype={'img_name': str})

# Apply the scaling function to each row of the DataFrame
scaled_data = scaled_data.apply(lambda row: scale_bbox(row, image_dir), axis=1)
scaled_data.to_csv('bbox_data_scaled.csv', index=False)
scaled_data.head()


,img_name,x_min,y_min,x_max,y_max,label,scaled_size,actual_size
0,00000018,29.118519,169.901154,84.446719,276.514942,unknown,"(750, 1333)","(285, 380)"
1,00000018,86.850668,50.791028,160.615367,158.934734,unknown,"(750, 1333)","(285, 380)"
2,00000018,30.860504,233.492178,58.715473,277.912354,unknown,"(750, 1333)","(285, 380)"
3,00000018,40.825769,191.754954,130.273464,286.123736,unknown,"(750, 1333)","(285, 380)"
4,00000018,28.236081,163.481472,103.817682,283.859392,person,"(750, 1333)","(285, 380)"


In [60]:
scaled_data = scaled_data[scaled_data['img_name'].str.len() >= 10]
scaled_data.to_csv('bbox_data_scaled.csv', index=False)

scaled_data

,img_name,x_min,y_min,x_max,y_max,label,scaled_size,actual_size
5,000000000139,-34.309862,407.299878,82.574335,425.321062,unknown,"(800, 1201)","(426, 640)"
6,000000000139,17.275577,417.012518,525.628856,429.082962,unknown,"(800, 1201)","(426, 640)"
7,000000000139,-0.428336,76.326047,3.251317,328.314845,unknown,"(800, 1201)","(426, 640)"
8,000000000139,599.953479,372.585724,638.359980,424.054777,unknown,"(800, 1201)","(426, 640)"
9,000000000139,-0.245864,375.194175,6.168726,423.172425,unknown,"(800, 1201)","(426, 640)"
...,...,...,...,...,...,...,...,...
32908,000000581781,11.741699,351.056465,265.509049,478.230396,unknown,"(800, 1071)","(478, 640)"
32909,000000581781,364.439186,429.945286,452.602144,480.675784,unknown,"(800, 1071)","(478, 640)"
32910,000000581781,32.025218,-0.129598,107.172034,16.083565,unknown,"(800, 1071)","(478, 640)"
32911,000000581781,-1.228728,217.778473,17.414693,309.652941,unknown,"(800, 1071)","(478, 640)"
